In [1]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster("local").setAppName("reduction-op")
sc = SparkContext(conf=conf)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/15 15:25:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Reduce
- `RDD.reduce(<task>)`
- 사용자가 지정하는 task를 받아 여러 개의 값을 하나로 줄이기

In [2]:
from operator import add

In [3]:
sample_rdd = sc.parallelize([1, 2, 3, 4, 5]).reduce(add) # add : lambda x, y : x + y
sample_rdd

15

## 파티션에 따라 결과물 달라짐
- 분산된 파티션들의 연산과 합치는 부분을 나눠서 생각하기

```python
lambda x, y : (x*2)+y
```

In [4]:
# 파티션이 1개인 경우
sc.parallelize([1, 2, 3, 4]).reduce(lambda x, y : (x*2)+y)

26

In [5]:
# 파티션이 2개
sc.parallelize([1, 2, 3, 4], 2).reduce(lambda x, y : (x*2)+y)

18

In [6]:
# 파티션이 3개
sc.parallelize([1, 2, 3, 4], 3).reduce(lambda x, y : (x*2)+y)

18

In [7]:
# 파티션이 4개
sc.parallelize([1, 2, 3, 4], 4).reduce(lambda x, y : (x*2)+y)

26

# Fold
- `RDD.fold(zeroValue, <task>)`
- `reduce`와 유사하나, 시작값을 zeroValue에 넣어놓고 reduce

In [8]:
rdd = sc.parallelize([2, 3, 4], 4)

In [9]:
print(rdd.reduce(lambda x, y : (x*y))) # (2*3)*4
print(rdd.fold(1, lambda x, y : (x*y)))

24
24


In [10]:
print(rdd.reduce(lambda x, y : x+y))

9


In [11]:
print(rdd.fold(1, lambda x, y : x+y))

14


# Aggregate (Action)
- `RDD.aggregate(zeroValue, seq0p, comb0p)`
    - `zeroValue` : 각 파티션에서 누적할 시작값
    - `seq0p` : 타입 변경 함수. 파티션 내부에서 사용할 task
    - `comb0p` : 합치는 함수. 파티션끼리 사용할 task

In [12]:
rdd = sc.parallelize([1, 2, 3, 4], 2)

In [13]:
seq0p = (lambda x, y : (x[0]+y, x[1]+1)) # 파티션 내 연산
comb0p = (lambda x, y : (x[0]+y[0], x[1]+y[1])) # 파티션의 모든 결과를 최종 연산

In [14]:
rdd.aggregate((0, 0), seq0p, comb0p)

(10, 4)

In [15]:
sc.stop()